In [ ]:
from dataclasses import dataclass, field
import numpy as np
import numpy.typing as npt
from scipy.fftpack import fft, ifft, fftshift, ifftshift, fftfreq
from scipy.constants import pi, c, h
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.signal import correlate
from scipy.optimize import curve_fit

rcParams['figure.dpi'] = 300
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['lines.linewidth'] = 3










def get_freq_range_from_time(time_s: npt.NDArray[float]
                             ) -> npt.NDArray[float]:
    """
    Calculate frequency range for spectrum based on time basis.

    When plotting a discretized pulse signal as a function of time,
    a time range is needed. To plot the spectrum of the pulse, one
    can compute the FFT and plot it versus the frequency range
    calculated by this function

    Parameters
    ----------
    time_s : npt.NDArray[float]
        Time range in seconds.

    Returns
    -------
    npt.NDArray[float]
        Frequency range in Hz.

    """
    return fftshift(fftfreq(len(time_s), d=time_s[1] - time_s[0]))


def get_time_from_freq_range(frequency_Hz: npt.NDArray[float]
                             ) -> npt.NDArray[float]:
    """
    Calculate time range for pulse based on frequency range.

    Essentially the inverse of the get_freq_range_from_time function.
    If we have a frequency range and take the iFFT of a spectrum field
    to get the pulse field in the time domain, this function provides the
    appropriate time range.

    Parameters
    ----------
    frequency_Hz : npt.NDArray[float]
        Freq. range in Hz.

    Returns
    -------
    time_s : npt.NDArray[float]
        Time range in s.

    """

    time_s = fftshift(fftfreq(len(frequency_Hz),
                              d=frequency_Hz[1] - frequency_Hz[0]))
    return time_s


def get_spectrum_from_pulse(time_s: npt.NDArray[float],
                            pulse_field: npt.NDArray[complex],
                            FFT_tol: float = 1e-3) -> npt.NDArray[complex]:
    """

    Parameters
    ----------
    time_s : npt.NDArray[float]
        Time range in seconds.
    pulse_field: npt.NDArray[complex]
        Complex field of pulse in time domain in units of sqrt(W).
    FFT_tol : float, optional
        When computing the FFT and going from temporal to spectral domain, the
        energy (which theoretically should be conserved) cannot change
        fractionally by more than FFT_tol. The default is 1e-7.

    Returns
    -------
    spectrum_field : npt.NDArray[complex]
        Complex spectral field in units of sqrt(J/Hz).

    """

    pulseEnergy = get_energy(time_s, pulse_field)  # Get pulse energy
    f = get_freq_range_from_time(time_s)
    dt = time_s[1] - time_s[0]

    assert dt > 0, (f"ERROR: dt must be positive, "
                    f"but {dt=}. {time_s[1]=},{time_s[0]=}")
    spectrum_field = ifftshift(
        ifft(ifftshift(pulse_field))) * (dt*len(f))  # Do shift and take fft
    spectrumEnergy = get_energy(f, spectrum_field)  # Get spectrum energy

    err = np.abs((pulseEnergy / spectrumEnergy - 1))


    assert (
        err < FFT_tol
    ), (f"ERROR = {err:.3e} > {FFT_tol:.3e} = FFT_tol : Energy changed "
        "when going from Pulse to Spectrum!!!")

    return spectrum_field


def get_pulse_from_spectrum(frequency_Hz: npt.NDArray[float],
                            spectrum_field: npt.NDArray[complex],
                            FFT_tol: float = 1e-3) -> npt.NDArray[complex]:
    """
    Converts the spectral field of a signal in the freq. domain temporal
    field in time domain

    Uses the iFFT to shift from freq. to time domain and ensures that energy
    is conserved

    Parameters
    ----------
    frequency_Hz : npt.NDArray[float]
        Frequency in Hz.
    spectrum_field : npt.NDArray[complex]
        Spectral field in sqrt(J/Hz).
    FFT_tol : float, optional
        Maximum fractional change in signal
        energy when doing FFT. The default is 1e-7.

    Returns
    -------
    pulse : npt.NDArray[complex]
        Temporal field in sqrt(W).

    """

    spectrumEnergy = get_energy(frequency_Hz, spectrum_field)

    time = get_time_from_freq_range(frequency_Hz)
    dt = time[1] - time[0]

    pulse = fftshift(fft(fftshift(spectrum_field))) / (dt*len(time))
    pulseEnergy = get_energy(time, pulse)

    err = np.abs((pulseEnergy / spectrumEnergy - 1))



    assert (
        err < FFT_tol
    ), (f"ERROR = {err:.3e} > {FFT_tol:.3e} = FFT_tol : Energy changed too "
        "much when going from Spectrum to Pulse!!!")
    return pulse


def get_energy(
    time_or_freq: npt.NDArray[float],
    field_in_time_or_freq_domain: npt.NDArray[complex],
) -> float:
    """
    Computes energy of signal or spectrum

    Gets the power or PSD of the signal from
    get_power(field_in_time_or_freq_domain)
    and integrates it w.r.t. either time or
    frequency to get the energy.

    Parameters
    ----------
    time_or_freq : npt.NDArray[float]
        Time range in seconds or freq. range in Hz.
    field_in_time_or_freq_domain : npt.NDArray[complex]
        Temporal field in [sqrt(W)] or spectral field [sqrt(J/Hz)].

    Returns
    -------
    energy: float
        Signal energy in J .

    """
    energy = np.trapz(
        get_power(field_in_time_or_freq_domain), time_or_freq)
    return energy

def get_power(field_in_time_or_freq_domain: npt.NDArray[complex]
              ) -> npt.NDArray[float]:
    """
    Computes temporal power or PSD

    For a real electric field, power averaged over an optical cycle is

    P = 1/T int_0^T( E_real**2 )dt.

    For a complex electric field, this same power is calculated as

    P = 0.5*|E|**2.

    Using the complex field makes calculations easier and the factor of
    0.5 is simply absorbed into the nonlinear parameter, gamma.
    Same thing works in the frequency domain.

    Parameters
    ----------
    field_in_time_or_freq_domain : npt.NDArray[complex]
        Temporal or spectral field.

    Returns
    -------
    power : npt.NDArray[complex]
        Temporal power (W) or PSD (J/Hz) at any instance or frequency.

    """
    power = np.abs(field_in_time_or_freq_domain) ** 2
    return power

def get_average(time_or_freq: npt.NDArray[float],
                pulse_or_spectrum: npt.NDArray[complex]) -> float:
    """
    Computes central time (or frequency) of pulse (spectrum)

    Computes central time (or frequency) of pulse (spectrum) by calculating
    the 'expectation value'.

    Parameters
    ----------
    time_or_freq : npt.NDArray[float]
        Time range in seconds or freq. range in Hz.
    pulse_or_spectrum : npt.NDArray[complex]
        Temporal or spectral field.

    Returns
    -------
    meanValue : float
        average time or frequency.

    """

    E = get_energy(time_or_freq, pulse_or_spectrum)
    meanValue = np.trapz(
        time_or_freq * get_power(pulse_or_spectrum), time_or_freq) / E
    return meanValue


def get_variance(time_or_freq: npt.NDArray[float],
                 pulse_or_spectrum: npt.NDArray[complex]) -> float:
    """
    Computes variance of pulse or spectrum

    Computes variance of pulse in time domain or freq domain via
    <x**2>-<x>**2

    Parameters
    ----------
    time_or_freq : npt.NDArray[float]
        Time range in seconds or freq. range in Hz.
    pulse_or_spectrum : npt.NDArray[complex]
        Temporal or spectral field.

    Returns
    -------
    variance : float
        variance in time or frequency domains.

    """
    E = get_energy(time_or_freq, pulse_or_spectrum)
    variance = (
        np.trapz(time_or_freq ** 2 *
                 get_power(pulse_or_spectrum), time_or_freq) / E
        - (get_average(time_or_freq, pulse_or_spectrum)) ** 2
    )
    return variance


def get_stdev(time_or_freq: npt.NDArray[float],
              pulse_or_spectrum: npt.NDArray[complex]) -> float:
    """
    Computes standard deviation of pulse or spectrum

    Computes std of pulse in time domain or freq domain via
    sqrt(<x**2>-<x>**2)

    Parameters
    ----------
    time_or_freq : npt.NDArray[float]
        Time range in seconds or freq. range in Hz.
    pulse_or_spectrum : npt.NDArray[complex]
        Temporal or spectral field.

    Returns
    -------
    stdev : float
        Stdev in time or frequency domains.

    """

    stdev = np.sqrt(get_variance(time_or_freq, pulse_or_spectrum))
    return stdev

def get_phase(pulse: npt.NDArray[complex]) -> npt.NDArray[float]:
    """
    Gets the phase of the pulse from its complex angle

    Calcualte phase by getting the complex angle of the pulse,
    unwrapping it and centering on middle entry.

    Parameters
    ----------
    pulse : npt.NDArray[complex]
        Complex electric field envelope in time domain.

    Returns
    -------
    phi : npt.NDArray[float]
        Phase of pulse at every instance in radians.

    """

    phi = np.unwrap(np.angle(pulse))  # Get phase starting from 1st entry
    phi = phi - phi[int(len(phi) / 2)]  # Center phase on middle entry
    return phi

def calculate_RMS(x):
    return np.sqrt( np.sum((x-np.mean(x))**2)/len(x)  )

In [ ]:
#Set up simulation parameters
dt_s = 0.1e-12 #Time resolution
N=10000 #Number of time steps = 2N+1 
t_s = np.arange(-N,N+1,1)*dt_s #Time range
N_steps = len(t_s)
idx_mix = int(N_steps/2)
f_Hz=get_freq_range_from_time(t_s) #2-sided Frequency range
f_pos_Hz=f_Hz[idx_mix:]   #1-sided Frequency range including f=0Hz



#Baseline CW field envelope is constant
field = np.ones_like(t_s)*1.0

#Set up random phase walk
step_size = 2*pi/(45)    #Moderate phase noise

#Generate several random walks 
N_walks = 100
random_phase_walk_list=np.zeros((N_walks,len(t_s)))
random_phase_walk_list_PM=np.zeros((N_walks,len(t_s)))

f_inst_RW_list=np.zeros((N_walks,len(t_s)))
f_inst_PM_list=np.zeros((N_walks,len(t_s)))


power_spectrum_RW_list=np.zeros_like(random_phase_walk_list)
power_spectrum_PM_list=np.zeros_like(random_phase_walk_list)

average_spectrum = np.zeros_like(field)*1.0
average_spectrum_PM = np.zeros_like(field)*1.0

average_phase_noise_spectrum = np.zeros_like(field)*1.0
average_phase_noise_spectrum_PM = np.zeros_like(field)*1.0

average_freq_noise_spectrum_RW = np.zeros_like(field)*1.0
average_freq_noise_spectrum_PM = np.zeros_like(field)*1.0

phi_RMS_avg = 0.0
phi_RMS_avg_PM = 0.0


PM_modulation_freq_Hz=30e9  
PM_amplitude_rad = 1.5  

for idx in range(N_walks):
    
    random_phase_walk = np.cumsum( np.random.choice([-1,1], size=len(t_s)))*step_size 
    
    random_phase_walk -= np.mean(random_phase_walk) #Subtract average phase
    phase_mod = PM_amplitude_rad*np.cos(2*pi*PM_modulation_freq_Hz*t_s+np.random.uniform(0,2*np.pi)) #Cosine function for phase modulation
    random_phase_walk_PM = random_phase_walk+phase_mod-np.mean(phase_mod) #Add phase modulation and subtract average
    random_phase_walk_list[idx,:]=random_phase_walk
    random_phase_walk_list_PM[idx,:]=random_phase_walk_PM 
    
    f_inst_RW=-np.gradient(random_phase_walk,t_s)/2/pi #Instantaneous frequency for random walk
    f_inst_PM=-np.gradient(random_phase_walk_PM,t_s)/2/pi #Instantaneous frequency for random walk + PM
    f_inst_RW_list[idx,:]=f_inst_RW - np.mean(f_inst_RW) #Add to list
    f_inst_PM_list[idx,:]=f_inst_PM - np.mean(f_inst_PM)
    
    average_freq_noise_spectrum_RW+=get_power(get_spectrum_from_pulse(t_s,f_inst_RW)) #Compute |FT{f_inst(t)}|**2 and add to average
    average_freq_noise_spectrum_PM+=get_power(get_spectrum_from_pulse(t_s,f_inst_PM))
    
    phi_RMS_avg += calculate_RMS(random_phase_walk)**2 #add up phi_RMS**2 so we can compute sqrt(<phi_RMS**2>) later
    phi_RMS_avg_PM += calculate_RMS(random_phase_walk_PM)**2
    
    
    phase_noise_spectrum = get_power(get_spectrum_from_pulse(t_s,random_phase_walk))  #Compute |FT{phi(t)}|**2 and add to average
    phase_noise_spectrum_PM = get_power(get_spectrum_from_pulse(t_s,random_phase_walk_PM))

    average_phase_noise_spectrum += phase_noise_spectrum
    average_phase_noise_spectrum_PM += phase_noise_spectrum_PM

    random_phase_walk_exp = np.exp(1j*random_phase_walk) 
    random_phase_walk_exp_PM = np.exp(1j*random_phase_walk_PM)
    
    power_spectrum_RW = get_power( get_spectrum_from_pulse(t_s,random_phase_walk_exp)  ) #Compute |FT{exp(i*phi(t))}|^2 to get spectrum of optical power
    power_spectrum_PM = get_power( get_spectrum_from_pulse(t_s,random_phase_walk_exp_PM)  )

    power_spectrum_RW_list[idx,:]=power_spectrum_RW
    power_spectrum_PM_list[idx,:]=power_spectrum_PM

    average_spectrum+=power_spectrum_RW
    average_spectrum_PM+=power_spectrum_PM
    

    
    
#Divide by number of walks to get average
average_spectrum/=N_walks
average_spectrum_PM/=N_walks

average_phase_noise_spectrum/=N_walks
average_phase_noise_spectrum_PM/=N_walks

average_freq_noise_spectrum_RW/=N_walks
average_freq_noise_spectrum_PM/=N_walks


phi_RMS_avg=np.sqrt(phi_RMS_avg/N_walks)
phi_RMS_avg_PM=np.sqrt(phi_RMS_avg_PM/N_walks)





In [ ]:
walk_idx = 0
phi    = random_phase_walk_list[0]
phi_PM = random_phase_walk_list_PM[0]

fig, ax = plt.subplots()
ax.set_title("Random phase walk.")
ax.plot(t_s/1e-12,phi,color='C7',alpha=0.7,label = "Pure random walk")
ax.plot(t_s/1e-12,phi_PM-10,color='C8',alpha=0.7,label=f"RW + {PM_modulation_freq_Hz/1e9:.1f}GHz PM")
#ax.legend()
ax.set_xlabel('Time [ps]')
ax.set_ylabel('$\phi(t)$ [rad]')   
#ax.set_ylim(-30,30)
ax.legend()
#plt.savefig('Random_phase_walk.svg')
plt.show()


#Plot absolute square of spectrum of electric field for one particular random walk
fig, ax = plt.subplots()
ax.set_title("$|FT[E(t)]|^2$")
ax.plot(f_Hz/1e9,power_spectrum_RW_list[0],color='C7',alpha=0.7)
ax.plot(f_Hz/1e9,power_spectrum_PM_list[0],color='C8',alpha=0.7)
Gamma=step_size**2/(2*np.pi*dt_s)
df=step_size**2/(2*np.pi*dt_s)
lor = Gamma/( (Gamma/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
#ax.plot(f_Hz/1e12, lor    ,color='r',alpha=0.7)
ax.set_xlabel('Freq [GHz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_xlim(-5*df/1e9,5*df/1e9)
ax.grid()
plt.show()

fig, ax = plt.subplots()
ax.set_title("$|FT[E(t)]|^2$")
ax.plot(f_Hz/1e9,power_spectrum_RW_list[0],color='C7',alpha=0.7)
ax.plot(f_Hz/1e9,power_spectrum_PM_list[0],color='C8',alpha=0.7)
Gamma=step_size**2/(2*np.pi*dt_s)
df=step_size**2/(2*np.pi*dt_s)
lor = Gamma/( (Gamma/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
#ax.plot(f_Hz/1e12, lor    ,color='r',alpha=0.7)
ax.set_xlabel('Freq [GHz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_xlim(-5*df/1e9,5*df/1e9)
ax.set_yscale('log')
ax.set_ylim(np.max(power_spectrum_RW_list[0])*1e-3,np.max(power_spectrum_RW_list[0])*3)
ax.grid()
plt.show()


In [ ]:
#Plot average spectrum and theoretical average spectrum (Lorentzian) 
#according to the Wiener-Khinchin Theorem    

fig, ax = plt.subplots()
ax.set_title("$<|FT[E(t)]|^2>$")
ax.plot(f_Hz/1e9,average_spectrum,color='C7',alpha=0.7)
ax.plot(f_Hz/1e9,average_spectrum_PM,color='C8',alpha=0.7)
Gamma=step_size**2/(2*np.pi*dt_s)
lor = Gamma/( (Gamma/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
ax.plot(f_Hz/1e9, lor    ,color='k',alpha=0.7)
ax.set_xlabel('Freq [GHz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_xlim(-4*df/1e9,4*df/1e9)
ax.grid()
plt.show()

#Same plot as above on a log scale
fig, ax = plt.subplots()
ax.set_title("$<|FT[E(t)]|^2>$")
ax.plot(f_Hz/1e9,average_spectrum,color='C7',alpha=0.7)
ax.plot(f_Hz/1e9,average_spectrum_PM,color='C8',alpha=0.7)
df=step_size**2/(2*np.pi*dt_s)
lor = df/( (df/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
ax.plot(f_Hz/1e9, lor    ,color='k',alpha=0.7)
ax.axvline(x=-PM_modulation_freq_Hz/1e9,color='C0')
ax.axvline(x=PM_modulation_freq_Hz/1e9,color='C0')
ax.set_xlabel('Freq [GHz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_yscale('log')
ax.set_xlim(-4*df/1e9,4*df/1e9)
ax.set_ylim(np.max(lor)*1e-3,np.max(lor)*3)
ax.grid()
plt.show()


#Same plot as above but 1-sided and on double-log scale
fig, ax = plt.subplots()
ax.set_title("$<|FT[E(t)]|^2>$")
ax.plot(f_Hz[int(len(f_Hz)/2):-1]/1e9,2*average_spectrum[int(len(f_Hz)/2):-1],color='C7',alpha=0.7)
ax.plot(f_Hz[int(len(f_Hz)/2):-1]/1e9,2*average_spectrum_PM[int(len(f_Hz)/2):-1],color='C8',alpha=0.7)
ax.axvline(x=PM_modulation_freq_Hz/1e9,color='C0')

df=step_size**2/(2*np.pi*dt_s)
lor = df/( (df/2)**2+(f_Hz)**2)/2/np.pi/(f_Hz[1]-f_Hz[0])
ax.plot(f_Hz[int(len(f_Hz)/2):-1]/1e9, 2*lor[int(len(f_Hz)/2):-1]    ,color='k',alpha=0.7)
ax.set_xlabel('Freq [GHz]')
ax.set_ylabel('Spectrum [norm.]')
ax.set_yscale('log')
ax.set_xscale('log')
ax.grid()
plt.show()

In [ ]:
#Plot particular random walks
phi=random_phase_walk_list[0]-np.mean(random_phase_walk_list[0])
phi_PM=random_phase_walk_list_PM[0]-np.mean(random_phase_walk_list_PM[0])

phi_RMS = calculate_RMS(phi)
phi_PM_RMS = calculate_RMS(phi_PM)

fig, ax = plt.subplots()
ax.set_title("Random phase walk. $\phi_{RMS}$="+f"{phi_RMS:.2f}rad")
ax.plot(t_s/1e-12,phi,color='C7',alpha=0.7)
ax.axhline(y=phi_RMS,color='C1',label='$\phi_{RMS}$')
ax.axhline(y=-phi_RMS,color='C1')
ax.legend()
ax.set_xlabel('Time [ps]')
ax.set_ylabel('$\phi(t)-\phi_{avg}$ [rad]')   
ax.set_ylim(-30,30)
plt.savefig('Random_phase_walk.svg')
plt.show()

#Plot phase noise spectrum, |FT(\phi(t))|^2, on log scale
freq_positive = f_Hz[int(len(f_Hz)/2+1):-1]
df=freq_positive[1]-freq_positive[0]
phase_spectrum = 2*get_power(get_spectrum_from_pulse(t_s,phi))[int(len(f_Hz)/2+1):-1]*df
phi_RMS_freq=np.sqrt(np.trapz(phase_spectrum,freq_positive)) #Determine phi_RMS by integrating spectrum in the frequency domain to show that it's conserved according to Plancherel's theorem

phase_spectrum_PM = 2*get_power(get_spectrum_from_pulse(t_s,phi_PM))[int(len(f_Hz)/2+1):-1]*df

fig, ax = plt.subplots()
ax.set_title("$|FT(\phi(t))|^2$. $\phi_{RMS}$ in time domain="+f"{phi_RMS:.3f}rad")
ax.plot(freq_positive,
        phase_spectrum,
        color='C7',
        alpha=0.7,
        label="$\phi_{RMS}$ in freq. domain="+f"{phi_RMS_freq:.2f}rad")
ax.plot(freq_positive,phase_spectrum_PM,color='C8',alpha=0.7)
ax.set_xlabel('Freq [Hz]')
ax.set_ylabel('[$rad^2$/Hz]')   
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlim(1e9,1e12)
ax.set_ylim(1e-16,1e-7)
ax.legend()
#ax.legend()
plt.show()

In [ ]:




#Plot phase noise spectrum, |FT(\phi(t))|^2, on log scale
df=freq_positive[1]-freq_positive[0]
phase_spectrum = 2*get_power(get_spectrum_from_pulse(t_s,phi))[int(len(f_Hz)/2+1):-1]*df
phi_RMS_freq=np.sqrt(np.trapz(phase_spectrum,freq_positive))

phase_spectrum_PM = 2*get_power(get_spectrum_from_pulse(t_s,phi_PM))[int(len(f_Hz)/2+1):-1]*df

fig, ax = plt.subplots()
ax.set_title("$<|FT(\phi(t))|^2>$")
ax.plot(freq_positive,phase_spectrum,color='C7',alpha=0.7)
ax.plot(freq_positive,
        2*average_phase_noise_spectrum[int(len(f_Hz)/2+1):-1]*df,
        color='k',
        alpha=0.7,
        label='Avg spectrum for RW')

ax.plot(freq_positive,phase_spectrum_PM,color='C8',alpha=0.7)
ax.plot(freq_positive,
        2*average_phase_noise_spectrum_PM[int(len(f_Hz)/2+1):-1]*df,
        color='r',
        alpha=0.7,
        label='Avg spectrum for RW+PM')

ax.set_xlabel('Freq [Hz]')
ax.set_ylabel('[$rad^2$/Hz]')   
ax.set_yscale('log')
ax.set_xlim(1e9,1e12)
ax.set_ylim(1e-16,1e-7)
ax.legend()
plt.show()


#Plot phase noise spectrum, |FT(\phi(t))|^2, on double-log
fig, ax = plt.subplots()
ax.set_title("$|FT(\phi(t))|^2$")
ax.plot(freq_positive,phase_spectrum,color='C7',alpha=0.7)
ax.plot(freq_positive,
        2*average_phase_noise_spectrum[int(len(f_Hz)/2+1):-1]*df,
        color='k',
        alpha=0.7,
        label='Avg spectrum for RW')

ax.plot(freq_positive,phase_spectrum_PM,color='C8',alpha=0.7)
ax.plot(freq_positive,
        2*average_phase_noise_spectrum_PM[int(len(f_Hz)/2+1):-1]*df,
        color='r',
        alpha=0.7,
        label='Avg spectrum for RW+PM')
ax.legend()
ax.set_xlabel('Freq [Hz]')
ax.set_ylabel('[$rad^2$/Hz]')   
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlim(1e9,1e12)
ax.set_ylim(1e-16,1e-7)
ax.grid()
plt.show()




In [ ]:
phi_RW=random_phase_walk_list[0]-np.mean(random_phase_walk_list[0])
#phi_Gauss = np.random.uniform(low=-23,high=23,size=len(phi_RW)) #Hot-swap the gaussian distribution with a uniform one to show that "flat" spectral behavior is a result of "memoryless" process and not the Gaussian!
phi_RMS_RW = calculate_RMS(phi_RW)

phi_Gauss = np.random.normal(loc=0,scale=phi_RMS_RW,size=len(phi_RW))
#phi_Gauss = np.random.normal(loc=0,scale=step_size*90,size=len(phi_RW))
phi_Gauss-=np.mean(phi_Gauss)
phi_RMS_Gauss = calculate_RMS(phi_Gauss)


print(phi_RMS_RW,phi_RMS_Gauss)

fig,ax=plt.subplots()
ax.set_title('$\phi(t)$')
ax.plot(t_s/1e-12,phi_Gauss,color='C0',alpha=0.7,label="$\phi_{RMS} = $"+f'{phi_RMS_Gauss:.1f}')
ax.plot(t_s/1e-12,phi_RW,color='C7',alpha=0.7,label="$\phi_{RMS} = $"+f'{phi_RMS_RW:.1f}')
ax.set_ylim(-4.5*phi_RMS_RW,4.5*phi_RMS_RW)
ax.set_xlabel('Time [ps]')
ax.set_ylabel('Phase [rad]')
ax.legend()
plt.show()

corr_RW=np.correlate(phi_RW, phi_RW, mode='same')
corr_Gauss=np.correlate(phi_Gauss, phi_Gauss, mode='same')

fig,ax=plt.subplots()
ax.set_title('Autocorrelation')
ax.plot(t_s/1e-12,corr_RW,color='C7',alpha=0.7)
ax.plot(t_s/1e-12,corr_Gauss,color='C0',alpha=0.7)
ax.set_xlabel('Time shift [ps]')
plt.show()

spec_corr_RW = get_spectrum_from_pulse(t_s,corr_RW)
spec_corr_Gauss = get_spectrum_from_pulse(t_s,corr_Gauss)

fig,ax=plt.subplots()
ax.set_title('$FT[AC[\phi(t)]]=|FT[\phi(t)]|^2$')
ax.plot(freq_positive/1e9,2*np.abs(spec_corr_RW[int(len(f_Hz)/2+1):-1])*df,color='C7',alpha=0.7)
ax.plot(freq_positive/1e9,2*np.abs(spec_corr_Gauss[int(len(f_Hz)/2+1):-1])*df,color='C0',alpha=0.7)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Freq. [GHz]')
ax.set_ylabel('[$rad^2$/Hz]')   
ax.grid()
plt.show()




In [ ]:
#Plot instantaneous frequency shift. Looks weird since we are taking derivative of random walk function where every step is +/- Delta_Phi
fig, ax = plt.subplots()
ax.set_title("Random frequency walk")
ax.plot(t_s/1e-12,f_inst_RW_list[0,:]/1e9,color='C7',alpha=0.7)
ax.set_xlabel('Time [ps]')
ax.set_ylabel('$f_{inst}(t)$ [GHz]')   
#plt.savefig('Random_freq_walk.svg')
ax.set_xlim(-10,10)
plt.show()


#Plot freq. noise spectrum. 
fig, ax = plt.subplots()
ax.set_title("Freq. Noise Spectrum (RW)")
phi_RW=random_phase_walk_list[0]-np.mean(random_phase_walk_list[0])
phase_spectrum_RW = 2*get_power(get_spectrum_from_pulse(t_s,phi_RW))[int(len(f_Hz)/2+1):-1]*df
phase_spectrum_PM = 2*get_power(get_spectrum_from_pulse(t_s,phi_PM))[int(len(f_Hz)/2+1):-1]*df
phase_spectrum_Gauss = 2*get_power(get_spectrum_from_pulse(t_s,phi_Gauss))[int(len(f_Hz)/2+1):-1]*df

freq_spectrum_RW=2*get_power(get_spectrum_from_pulse(t_s,f_inst_RW_list[0,:]))[int(len(f_Hz)/2+1):-1]*df
freq_spectrum_PM=2*get_power(get_spectrum_from_pulse(t_s,f_inst_PM_list[0,:]))[int(len(f_Hz)/2+1):-1]*df


#ax.plot(freq_positive[1:]/1e9,freq_positive[1:]**2*phase_spectrum_RW[1:],color='k',alpha=0.7,label='$f^2|FT[\phi(t)]|^2$')
#ax.plot(freq_positive[1:]/1e9,                     freq_spectrum_RW[1:] ,color='C7',alpha=0.7,label='$|FT[f_{inst}(t)]|^2$')

ax.plot(freq_positive[1:]/1e9,freq_positive[1:]**2*phase_spectrum_PM[1:],color='C8',alpha=0.7,label='$f^2|FT[\phi(t)]|^2$')
ax.plot(freq_positive[1:]/1e9,freq_positive[1:]**2*phase_spectrum_Gauss[1:],color='C0',alpha=0.7,label='$f^2|FT[\phi(t)]|^2$')
ax.set_xlabel('Freq [GHz]')
ax.set_ylabel('$[Hz^2/Hz]$')   
ax.set_yscale('log')
ax.set_xscale('log')
#ax.legend()
plt.show()
